# Parte 1: Replicando experimentos de ....

__Proposta:__ For additional improvement of the prediction of a classifier, we suggest combing bagging and boosting methodology with sum rule voting (Vote B&B)

__Metodologia__:  In order to calculate the classifiers’ accuracy, the whole training set was divided into ten mutually exclusive and equal-sized subsets and for each subset the classifier was trained on the union of all of the other subsets. Then, cross validation was run 10 times for each algorithm and the median value of the 10-cross validations was calculated.

__Bases de dados__: we used 36 well-known datasets mainly from many domains from the UCI repository.  we 
have used data sets from the domains of: pattern recognition (anneal, iris, mushroom, zoo), image recognition (ionosphere, sonar), medical diagnosis (breast-cancer, breast-w, colic, diabetes, heart-c, heart-h, heart-statlog, hepatitis, lymphotherapy, primary-tumor) commodity trading (autos, credit-g) music composition (waveform), computer games (kr-vs-kp, monk1, monk2), various control applications (balance), language morphological analysis (dimin) [9] and prediction of student dropout (student) [10].

__sum rule voting:__ When the sum rule is used each sub-ensemble has to give a confidence value for each 
candidate. In our algorithm, voters express the degree of their preference using as confidence score the probabilities of sub-ensemble prediction. Next all confidence values are added for each candidate and the candidate with the highest sum wins the election.

__sub-ensembles:__

__sub-classifiers:__ We used 10 sub-classifiers for each sub-ensemble for the proposed algorithm. 

In [23]:
import pandas as pd
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score

## Dataset: waveform

In [2]:
dataset = pd.read_csv("../data/waveform.data",header=None)
#dataset.columns[-1] = 'TARGET'
dataset = dataset.rename(columns={dataset.columns[-1]: 'TARGET'})

print(dataset.shape)
print(dataset['TARGET'].value_counts())

dataset.head()

(5000, 22)
2    1696
0    1657
1    1647
Name: TARGET, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,TARGET
0,-1.23,-1.56,-1.75,-0.28,0.60,2.22,0.85,0.21,-0.20,0.89,...,2.89,7.75,4.59,3.15,5.12,3.32,1.20,0.24,-0.56,2
1,-0.69,2.43,0.61,2.08,2.30,3.25,5.52,4.55,2.97,2.22,...,1.24,1.89,1.88,-1.34,0.83,1.41,1.78,0.60,2.42,1
2,-0.12,-0.94,1.29,2.59,2.42,3.55,4.94,3.25,1.90,2.07,...,2.50,0.12,1.41,2.78,0.64,0.62,-0.01,-0.79,-0.12,0
3,0.86,0.29,2.19,-0.02,1.13,2.51,2.37,5.45,5.45,4.84,...,2.58,1.40,1.24,1.41,1.07,-1.43,2.84,-1.18,1.12,1
4,1.16,0.37,0.40,-0.59,2.66,1.00,2.69,4.06,5.34,3.53,...,4.30,1.84,1.73,0.21,-0.18,0.13,-0.21,-0.80,-0.68,1


## Modelos

In [3]:
def evaluate(dataset, model, model_name, params={}):
    results = dict()

    scores = list()
    splits = KFold(n_splits=10).split(dataset)

    for i, (train_idx, test_idx) in enumerate(splits):
        print(f"Fold {i+1}/10")
        X_train = dataset.iloc[train_idx,~dataset.columns.isin(["TARGET"])]
        X_test = dataset.iloc[test_idx,~dataset.columns.isin(["TARGET"])]

        y_train = np.ravel(dataset.iloc[train_idx].loc[:,"TARGET"])
        y_test = np.ravel(dataset.iloc[test_idx].loc[:,"TARGET"])

        clf = model(**params)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        scores.append(accuracy_score(y_test, y_pred))
    results[model_name] = scores
    return results

In [62]:
res_decisiontree = evaluate(dataset, 
                            DecisionTreeClassifier, 
                            'DecisionTree', 
                            {'max_depth' : np.sqrt(X_train.shape[1])})

Fold 1/10
Fold 2/10
Fold 3/10
Fold 4/10
Fold 5/10
Fold 6/10
Fold 7/10
Fold 8/10
Fold 9/10
Fold 10/10


In [53]:
res_randomforest = evaluate(dataset, 
                            RandomForestClassifier, 
                            'RandomForest', 
                            {'n_estimators' : 25, 'max_depth' : np.sqrt(X_train.shape[1])})

Fold 1/10
Fold 2/10
Fold 3/10
Fold 4/10
Fold 5/10
Fold 6/10
Fold 7/10
Fold 8/10
Fold 9/10
Fold 10/10


In [57]:
res_adaboost = evaluate(dataset, 
                        AdaBoostClassifier, 
                        'Adaboost', 
                        {
                            'n_estimators' : 25, 
                            'base_estimator' : DecisionTreeClassifier(max_depth=np.sqrt(X_train.shape[1]))
                        })

Fold 1/10
Fold 2/10
Fold 3/10
Fold 4/10
Fold 5/10
Fold 6/10
Fold 7/10
Fold 8/10
Fold 9/10
Fold 10/10


In [59]:
def evaluate_bb(dataset):
    results = dict()

    scores = list()
    splits = KFold(n_splits=10).split(dataset)

    for i, (train_idx, test_idx) in enumerate(splits):
        print(f"Fold {i+1}/10")
        
        probs_1 = list()
        probs_2 = list()
    
        X_train = dataset.iloc[train_idx,~dataset.columns.isin(["TARGET"])]
        X_test = dataset.iloc[test_idx,~dataset.columns.isin(["TARGET"])]

        y_train = np.ravel(dataset.iloc[train_idx].loc[:,"TARGET"])
        y_test = np.ravel(dataset.iloc[test_idx].loc[:,"TARGET"])

        # Modelo Bagging : Random forest
        modelo_rf = RandomForestClassifier(n_estimators=10, 
                                           max_depth=np.sqrt(X_train.shape[1]))
        modelo_rf.fit(X_train, y_train)

        for i in range(modelo_rf.n_estimators):
            prob = modelo_rf.estimators_[i].predict_proba(X_test)
            probs_1.append(prob)

        # Modelo boosting : Ada boost
        modelo_ada = AdaBoostClassifier(n_estimators=10, 
                                        base_estimator=DecisionTreeClassifier(max_depth=np.sqrt(X_train.shape[1])))
        modelo_ada.fit(X_train, y_train)

        for i in range(modelo_ada.n_estimators):
            prob = modelo_ada.estimators_[i].predict_proba(X_test)
            probs_2.append(prob)

        # Sum rule vote
        total_probs = probs_1 + probs_2
        baselearner_matrix = np.sum(total_probs, axis=0)
        y_pred = np.argmax(baselearner_matrix,axis=1)

        # Metrics
        scores.append(accuracy_score(y_test, y_pred))
    results['vote_B&B'] = scores
    return results

In [60]:
res_voteBB = evaluate_bb(dataset)

Fold 1/10
Fold 2/10
Fold 3/10
Fold 4/10
Fold 5/10
Fold 6/10
Fold 7/10
Fold 8/10
Fold 9/10
Fold 10/10


__Comparando resultados: base de dados Waveform__

In [65]:
print(f"{'DecisionTree'} : {np.mean(*res_decisiontree.values())}")
print(f"{'RandomForest'} : {np.mean(*res_randomforest.values())}")
print(f"{'Adaboot'} : {np.mean(*res_adaboost.values())}")
print(f"{'vote B&B'} : {np.mean(*res_voteBB.values())}")

DecisionTree : 0.7376
RandomForest : 0.8222000000000002
Adaboot : 0.8246
vote B&B : 0.8301999999999999


# Parte 2: Sugerindo alteração

__Opção 1:__ sugerir uma estratégia diferente para a combinação dos ensembles. (Trocar o `Sum rule voting`)

__Opção 2:__ utilizar algum algoritmo de redes neurais com base learner para a combinação dos ensembles.